# Setup

In [ ]:
import pickle as pkl
from context_descriptor import ContextDescriptor
from prompt_generator import PromptGenerator
from model_request_handler import ModelRequestHandler
import numpy as np
from examples_manager import Examples_manager
from sklearn.metrics import classification_report
import time

In [2]:
DATASET = "EXTRASENSORY"
if DATASET not in ["DOMINO", "EXTRASENSORY"]:
    raise ValueError("DATASET name must be 'DOMINO' or 'EXTRASENSORY'")

In [3]:
extrasensory_classes = ['BICICLYNG', 'LAYING_DOWN', 'MOVING_BY_CAR', 'ON_TRANSPORT', 'SITTING', 'STANDING', 'WALKING']
domino_classes = ['BRUSHING_TEETH', 'CYCLING', 'ELEVATOR_DOWN', 'ELEVATOR_UP', 'LYING', 'MOVING_BY_CAR', 'RUNNING', 'SITTING', 'SITTING_ON_TRANSPORT', 'STAIRS_DOWN', 'STAIRS_UP', 'STANDING', 'STANDING_ON_TRANSPORT', 'WALKING']

In [4]:
if DATASET == "DOMINO":
    CLASSES = domino_classes
elif DATASET == "EXTRASENSORY":
    CLASSES = extrasensory_classes
else:
    CLASSES = None
    raise ValueError("DATASET name must be 'DOMINO' or 'EXTRASENSORY'")

# Load dataset

In [5]:
if DATASET == "DOMINO":
    with open("datasets/domino/domino_unique_contexts.pkl", "rb") as file:
        unique_contexts = pkl.load(file)
    with open("datasets/domino/domino_unique_contexts_ontology_consistencies.pkl", "rb") as file:
        unique_contexts_ontology_consistencies = pkl.load(file)
    with open("datasets/domino/domino_unique_contexts_labels.pkl", "rb") as file:
        unique_contexts_labels = pkl.load(file)

elif DATASET == "EXTRASENSORY":
    with open("datasets/extrasensory/extrasensory_unique_contexts.pkl", "rb") as file:
        unique_contexts = pkl.load(file)
    with open("datasets/extrasensory/extrasensory_unique_contexts_labels.pkl", "rb") as file:
        unique_contexts_labels = pkl.load(file)
    with open("datasets/extrasensory/extrasensory_unique_contexts_ontology_consistencies.pkl", "rb") as file:
        unique_contexts_ontology_consistencies = pkl.load(file)
        
else:
    raise ValueError("DATASET name must be 'DOMINO' or 'EXTRASENSORY'")

# Define system message

In [6]:
system_message = f'''
    You are a domain expert in charge of checking which human activities are suitable with respect to the current surrounding context of a user. 
    You will receive as input a description of the surrounding context of the user. You have to determine which are the activities that are suitable in the given context. 
    This is the list of the possible user activities that you have to consider: {', '.join(CLASSES)}

    Accomplish the task by following the steps below:

    Step 1: Analyze the context.
    Step 2: Determine which activities are likely in the given context, by analyzing each activity. Apply the open-world assumption: anything that is not explicit in the surrounding context of the user may be possible. Do not exclude activities for which you cannot determine their likelihood or the ones for which there are no specific information about objects and vehicles that the user is using.
    Step 3: Provide the result in the form of a list.'''

In [7]:
#Think about the "answer and step by step analyze each activity.
#system_message += "In the following i'll give you some examples of contexts that you can receive and the answer " \
#                  "that i would expect: "

# Initialize helper classes

In [8]:
contest_descriptor = ContextDescriptor()
prompt_generator = PromptGenerator(system_message)
request_handler = ModelRequestHandler()

# Initialize example manager

In [9]:
if DATASET == "DOMINO":
    example_manager = Examples_manager("domino_examples.csv")
elif DATASET == "EXTRASENSORY":
    example_manager = Examples_manager("extrasensory_examples.csv")
else:
    example_manager = None
    raise ValueError("DATASET name must be 'DOMINO' or 'EXTRASENSORY'")

# Choose context desciption function

In [10]:
if DATASET == "DOMINO":
    create_context_description = lambda context_vector: contest_descriptor.create_domino_context_description(context_vector)
elif DATASET == "EXTRASENSORY":
    create_context_description = lambda context_vector: contest_descriptor.create_extrasensory_context_description(context_vector)
else:
    create_context_description = None
    raise ValueError("DATASET name must be 'DOMINO' or 'EXTRASENSORY'")

# Make the api calls

In [ ]:
all_responses = []
i = 0
for c, l, o in list(zip(unique_contexts, unique_contexts_labels, unique_contexts_ontology_consistencies)):
    i += 1
    print(c)
    description = create_context_description(c)
    print('CURRENT: ', description)
    most_similar_examples = example_manager.get_most_similar_examples(description)
    #print('SIMILAR: \n', most_similar_examples) 
    print(description)
    prompt = prompt_generator.generate_prompt(description, most_similar_examples)
    print(prompt)
    responses = request_handler.handle_request(prompt, repetitions=1, return_all=True)
    print(responses)
    #all_responses.append(responses)
    print("---------------------------------------------------------------------------------")
    time.sleep(3)


# Dump answers
Increment n each time to have different files

In [12]:
n = 1

In [25]:
if DATASET == "DOMINO":
    answers_path = f"answers/domino/a_{n}.pkl"
elif DATASET == "EXTRASENSORY":
    answers_path = f"answers/extrasensory/a_{n}.pkl"
else:
    answers_path = None
    raise ValueError("DATASET name must be 'DOMINO' or 'EXTRASENSORY'")

In [26]:
with open(answers_path, "wb") as file:
    pkl.dump(all_responses, file)

# Load answers and ontology

In [13]:
if DATASET == "DOMINO":
    answers_path = f"answers/domino/a_{n}.pkl"
elif DATASET == "EXTRASENSORY":
    answers_path = f"answers/extrasensory/a_{n}.pkl"
else:
    answers_path = None
    raise ValueError("DATASET name must be 'DOMINO' or 'EXTRASENSORY'")

In [14]:
if DATASET == "DOMINO":
    ontology_path = f'datasets/domino/domino_unique_contexts_ontology_consistencies.pkl'
elif DATASET == "EXTRASENSORY":
    ontology_path = f"datasets/extrasensory/extrasensory_unique_contexts_ontology_consistencies.pkl"
else:
    answers_path = None
    raise ValueError("DATASET name must be 'DOMINO' or 'EXTRASENSORY'")

In [16]:
#with open(answers_path, "rb") as file:
#    all_responses = pkl.load(file)
with open(ontology_path, "rb") as file:
    ontology_consistencies = pkl.load(file)

# Compute results

In [36]:
def count_occurrences(activity, list_of_lists):
    count = 0
    for sublist in list_of_lists:
        count += sublist.count(activity)
    return count

In [37]:
def get_classes_from_binary_vector(bin_vec):
    classes_to_return = []
    for v, c in zip(bin_vec, CLASSES):
        if v==1:
            classes_to_return.append(c)
    return classes_to_return

In [39]:
min_count = 1

In [40]:
preds = []
for response in all_responses:
    # print(response)
    aggregated_response = []
    for act in CLASSES:
        if count_occurrences(act.lower(), response) >= min_count:
            aggregated_response.append(1)
        else:
            aggregated_response.append(0)
    preds.append(aggregated_response)

In [30]:
print(classification_report(ontology_consistencies, preds, target_names=CLASSES))

# Get results for DL

In [42]:
#TODO